In [ ]:
import pandas as pd
import numpy as np

from data.constants import WORLD_CRS
from data.divvy import DivvyClient
from data.geo import project_gdf

In [ ]:
bike_stations_file = "../data/raw/bike_stations.geojson"

In [42]:
client = DivvyClient()

# Pipeline in

(none)

# Bike Stations

In [43]:
bike_stations = client.s3_bike_stations()

DEBUG: populating bucket paths.


In [44]:
# # Some data is projected and some is not so we need to reproject to combine.
# from collections import Counter
# Counter([x.crs.to_authority() for x in bike_stations])
# # Actually only one of the files was projected so we will un-project everything.

In [45]:
bike_stations = [project_gdf(x, WORLD_CRS) for x in bike_stations]
# Now we can combine.
# note: station_id are floats on all normalized tables!
bike_stations = pd.concat(bike_stations, ignore_index=True).drop_duplicates()
null_geoms = (np.isnan(bike_stations[['longitude','latitude']]).any(axis=1) | 
              np.isinf(bike_stations[['longitude','latitude']]).any(axis=1) |
              np.isnan(bike_stations['geometry'].x) | 
              np.isinf(bike_stations['geometry'].x) | 
              np.isnan(bike_stations['geometry'].y) | 
              np.isinf(bike_stations['geometry'].y))
print("Dropping {} null geoms".format(null_geoms.sum()))
bike_stations = bike_stations[~null_geoms]
bike_stations = bike_stations.drop(columns=['latitude', 'longitude'])

Dropping 1 null geoms


# Pipeline out


In [46]:
bike_stations.to_file(bike_stations_file, index=False)